### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2022
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico 3 - Embeddings

Integrantes:
* Fernando Agustin Cardellino
* Adrián Zelaya

### Objetivos:

Esta notebook se enfoca en los siguientes ejercicios de la mentoría:

Práctico 3:
* Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

Práctico 4:
* Realizar el proceso utilizando Gensim-Doc2Vec. Generar un input texto.

Los vectores obtenidos en esta notebook serán reutilizados en el siguiente [repositorio](https://github.com/adrian-alejandro/autoML), para la optimización automática de hiperparámetros de modelos de ML en el marco de la materia AutoML.

In [104]:
# Importamos las librerías necesarias
import os
import itertools
import numpy as np
import pandas as pd
import swifter
import warnings

import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split

try:
    from nltk.corpus import stopwords
except:
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords

warnings.filterwarnings("ignore")

In [105]:
# Cargamos modelo de spacy y stopwords de NLTK
nlp = spacy.load("es_core_news_sm")

STOPWORDS_ES = stopwords.words('spanish')
BREAKPOINT=None  # None para analizar todos los documentos, sino un número para analizar hasta n documentos

In [106]:
# Ubicación de los documentos
CURR_DIR = os.getcwd()  # Gets current directory
embeddings_dir = os.path.join(CURR_DIR, "embeddings")
files_dir = os.path.join(CURR_DIR, "Documentos")

### Funciones que vamos a utilizar en el Práctico

#### Funciones auxiliares

In [107]:
def get_list_of_files(dirname, return_dir=False):
    """Returns a list of the files and subdirectories contained in a given input directory dirname
    """
    files = os.listdir(dirname)
    all_files = list()
    # Iterate over all the entries
    for file in files:
        # Create full path
        full_path = os.path.join(dirname, file)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(full_path):
            if return_dir:
                all_files.append(full_path.split(os.sep)[-1])
            else:
                all_files = all_files + get_list_of_files(full_path)
        else:
            all_files.append(full_path)

    return all_files

#### Funciones específicas del análisis

In [108]:
# Funciones de preprocesamiento

def initialize_dataset(files, sample_size=None):
    """Función que partir de un directorio o lista de archivos inicializa el dataset en un dataframe 
    con la información básica del corpus: 'archivo', 'fuero', 'path'
    """
    def get_filename(file_path):
        # extrae el nombre del archivo
        return file_path.split(os.sep)[-1]
    
    def get_fuero(file_path):
        # extrae el nombre del fuero a partir de la carpeta en donde reside el documento
        return file_path.split(os.sep)[-2]
    
    if os.path.isdir(files): # si el input is un directorio
        list_of_files = get_list_of_files(files)
    else:
        list_of_files = files 
    
    dataset = [{'archivo': get_filename(file), 'fuero': get_fuero(file), 'path': file} for file in list_of_files]
    if sample_size:
        try:
            return pd.DataFrame(dataset).sample(n=sample_size)
        except:
            pass
    return pd.DataFrame(dataset)

def preprocess_dataset(dataset, encoding='utf-8', object_type='word'):
    """Función que realiza las siguientes tareas de preprocesamiento de texto dado un dataframe de input:
    - extrae texto dada la ruta de un archivo
    - transforma el texto en tokens
    - limpia el texto: remueve no-palabras y stopwords, lematiza tokens y los transforma a minúsculas
    """
    
    REPLACEMENTS = [('á', 'a'), ('é', 'e'), ('í', 'i'), ('ó', 'o'), ('ú', 'u')]
    
    def get_text_from_file(file_path):
        with open(file_path, encoding=encoding) as f:
            return f.read()
        
    def extract_tokens(text, object_type=object_type):
        nlp_doc = nlp(text)
        return nlp_doc.ents if object_type == 'entity' else nlp_doc
    
    def preprocess_tokens(tokens):
        """Remueve tokens que no sean palabras ni stopwords, y los pasa a su versión lematizada y en minúsculas
        """
        try:
            return [replace_tokens(token.lemma_.lower() , replacements=REPLACEMENTS)
                    for token in tokens if token.is_alpha and not token.is_stop]
        except:
            return None
        
    def replace_tokens(token, replacements):
        aux = token
        for old, new in replacements:
            aux = aux.replace(old, new)
        return aux
        
    
    # Get text from files
    dataset['texto'] = dataset['path'].swifter.apply(lambda x: get_text_from_file(x))
    
    # Split text into tokens
    dataset['tokens'] = dataset['texto'].swifter.apply(lambda x: extract_tokens(x))
    
    # Clean tokens
    dataset['texto_clean'] = dataset['tokens'].swifter.apply(lambda x: preprocess_tokens(x))
    
    return dataset


def vectorize_corpus(vectorizer, corpus, to_array=True):
    """
    Función que vectoriza un corpus de palabras compuesto por documentos, utilizando cualquier vectorizador de 
    scikit-learn que sea ingresado por el usuario.
    Se asume que el corpus ya se encuentra tokenizado (mediante la función generar_corpus()), por lo que se 'anula'
    el tokenizador por defecto de los vectorizadores.
    """
    vectorizer_ = vectorizer(
        tokenizer=lambda doc: doc, # Pisamos el tokenizador para que tome los tokens como vienen (ver descripción)
        lowercase=False # Paso ya incluido en nuestro preprocesamiento
    )
    if to_array:
        return vectorizer_.fit_transform(corpus).toarray()
    return vectorizer_.fit_transform(corpus)

## Generamos el corpus y tokenizamos

In [109]:
n_docs = BREAKPOINT if not None else 'todos'
    
print(f"\nAnálisis para {n_docs} documentos\n")

# Inicializamos dataset
data = initialize_dataset(files_dir, sample_size=BREAKPOINT)

# Preprocesamos el dataset
corpus_df = preprocess_dataset(data)

display(corpus_df.head())


Análisis para None documentos



Pandas Apply:   0%|          | 0/243 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/243 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/243 [00:00<?, ?it/s]

,archivo,fuero,path,texto,tokens,texto_clean
0,9 BAEZ-FLECHA BUS.pdf.txt,LABORAL,/home/adrian/PycharmProjects/Busqueda-y-Recome...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"(SALA, LABORAL, -, TRIBUNAL, SUPERIOR, \n\n, P...","[sala, laboral, tribunal, superior, protocolo,..."
1,90 FUNES-COYSPU.pdf.txt,LABORAL,/home/adrian/PycharmProjects/Busqueda-y-Recome...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"(SALA, LABORAL, -, TRIBUNAL, SUPERIOR, \n\n, P...","[sala, laboral, tribunal, superior, protocolo,..."
2,1 QUINTEROS-CONSOLIDAR.pdf.txt,LABORAL,/home/adrian/PycharmProjects/Busqueda-y-Recome...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"(SALA, LABORAL, -, TRIBUNAL, SUPERIOR, \n\n, P...","[sala, laboral, tribunal, superior, protocolo,..."
3,3 SANGUEDOLCE-MUNICIPALIDAD DE VILLA ALLENDE.p...,LABORAL,/home/adrian/PycharmProjects/Busqueda-y-Recome...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"(SALA, LABORAL, -, TRIBUNAL, SUPERIOR, \n\n, P...","[sala, laboral, tribunal, superior, protocolo,..."
4,188 LUCIANO-NICOLAS.pdf.txt,LABORAL,/home/adrian/PycharmProjects/Busqueda-y-Recome...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"(SALA, LABORAL, -, TRIBUNAL, SUPERIOR, \n\n, P...","[sala, laboral, tribunal, superior, protocolo,..."


# Transformación del texto en vectores numéricos

Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

## CountVectorizer

Este vectorizador convierte una colección de textos en una matriz de recuentos de tokens.

Para ello, primero **tokeniza** las palabras y asigna un token id (número entero) a cada token. El tokenizador por default utiliza espacios y separadores de puntuación como separadores. Luego, **cuenta** las ocurrencias de los tokens en cada documento.

El producto de salida es una matriz rala/dispersa, donde cada fila representa un documento mientras que cada columna representa un token. Los valores de la matriz representan el número de ocurrencias del token en el documento.

In [110]:
vector_cv = vectorize_corpus(CountVectorizer, corpus_df['texto_clean'])

np.save(
    os.path.join(embeddings_dir, 'embedding_CV.npy'),
    vector_cv
)

## TfidfVectorizer

Este vectorizador convierte una colección de textos en una matriz de coeficientes **TF-IDF** (**T**erm-**F**requency - **I**nverse **D**ocument-**F**requency). Por la naturaleza del método, suele ser más útil en textos largos dado que para textos cortos los vectores resultantes pueden ser ruidosos/inestables. Es equivalente a utilizar el vectorizador `CountVectorizer` seguido del transformador `TfidfTransformer`.

Los coeficientes TF-IDF nos permiten balancear el peso de las palabras de acuerdo a su frecuencia de ocurrencia, dándole menos importancia a palabras que se repiten seguido y resaltando aquellas que son más inusuales/raras.

Para ello se aplica la siguiente transformación:

$\text{tf-idf}_{(t,d)} = \text{tf}_{(t,d)} * \text{idf}_{(t)}$ ,

donde $t$ corresponde a un término (token/palabra) y $d$ a un documento, además:

$\text{tf}_{(t,d)} = \frac{\text{ocurrencias de } t \text{ en } d }{\text{número de palabras en }d}$ $ $ y $ $  $\text{idf}_{(t)} = \log{\frac{1+n}{1+\text{df}_{(t)}}}$ ,

donde $n$ es el número total de documentos en el corpus y $\text{df}$ es el número de documentos que contienen el término/palabra $t$.

Finalmente, los vectores resultantes de cada documento, i.e. fila de la matriz, es normalizado utilizando la norma seleccionada (default: Euclideana o $L^2$):

$v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v{_1}^2 + v{_2}^2 + \dots + v{_n}^2}}$

In [111]:
vector_tfidf = vectorize_corpus(TfidfVectorizer, corpus_df['texto_clean'])

np.save(
    os.path.join(embeddings_dir, 'embedding_TFIDF.npy'),
    vector_tfidf
)

## HashingVectorizer

Este vectorizador convierte una colección de textos en una matriz de ocurrencias de tokens.

Se diferencia de `CountVectorizer` en que utiliza *feature hashing*, i.e. aplica una función de hash a cada token y usa dicho hash como índice en vez de buscar los índices en alguna tabla asociativa.

Esto hace que sea un método comparativamente mucho más rápido y con un uso reducido de memoria, útil para grandes datasets.

En cuanto a sus desventajas, como no guarda registro de las características de los inputs originales, no es posible aplicar la transformación inversa, lo cual implica que no es posible saber qué características (features) pueden ser más relevantes en el modelo. 

Otro inconveniente con el método es que no permite balancear los tokens según su frecuencia de ocurrencia (IDF), aunque esto se puede suplir incluyendo un `TfidfTransformer` en el pipeline.

Por otro lado, tokens distintos pueden ser mapeados al mismo índice (hash), aunque en la práctica esto rara vez ocurre dado que el número de atributos debe ser lo suficientemente grande, e.g. $2^{18}$ para problemas de clasificación de textos.

In [112]:
vector_HV = vectorize_corpus(HashingVectorizer, corpus_df['texto_clean'], to_array=False)

## Exportamos los datos

In [116]:
X = vector_tfidf
y = corpus_df.fuero.values

X_path = os.path.join(embeddings_dir, 'X.npy')
y_path = os.path.join(embeddings_dir, 'y.npy')
df_path = os.path.join(embeddings_dir, 'processed_dataset.csv')

np.save(
    X_path,
    X
)

np.save(
    y_path,
    y
)

corpus_df.to_csv(
    df_path,
    index=False,
    sep='|'    
)

Zipeamos para poder subir al repositorio

In [117]:
for path in [X_path, y_path, df_path]:
    zip_path = f"{path}.zip"
    !zip -r {zip_path} {path}

  adding: home/adrian/PycharmProjects/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/embeddings/X.npy (deflated 96%)
  adding: home/adrian/PycharmProjects/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/embeddings/y.npy (deflated 78%)
  adding: home/adrian/PycharmProjects/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/embeddings/processed_dataset.csv (deflated 75%)


# Referencias

* [How to Encode Text Data for Machine Learning with scikit-learn](https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/)
* [sckit-learn documentation - CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
* [sckit-learn user guide - Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) 
* [sckit-learn documentation - TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)
* [sckit-learn user guide - Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) 
* [sckit-learn documentation - HashingVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer)
* [sckit-learn user guide - Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#vectorizing-a-large-text-corpus-with-the-hashing-trick) 
* [Wikipedia - Feature hashing](https://en.wikipedia.org/wiki/Feature_hashing)